In [1]:
import pandas as pd

In [2]:
!pwd

/private/home/dlicht/flores101_data/2021_09_human_eval_sample_building


In [3]:
ls

'How to load FLORES data without PANDAS errors.ipynb'
'NLLB October FLORES sampling-batch.ipynb'
'NLLB October FLORES sampling.ipynb'
'NLLB build Cross-Lingual Calibration Set v1.ipynb'
'daiquery-unique flores IDs-2021-09-08 11_00am.tsv'


In [4]:
write_tsv = True

In [5]:
LANG3_list = [
#     'rus',
#     'deu',
#     'hau',
#     'isl',
#     'ces',
#     'jpn',
    'snd',
    'azj',
    'amh',
    'zul',
    'kat',
    'urd',
    'ara',
    'hin',
    'slv',
    'swh',
    'bul',
    'bos',
    'ron',
    'por',

#     'zho_simpl',
#     'fra',
#     'kor',
]

LANG2_list = [
#     'ru',
#     'de',
#     'ha',
#     'is',
#     'cs',
#     'ja',
    'sd',
    'az',
    'am',
    'zu',
    'ka',
    'ur',
    'ar',
    'hi',
    'sl',
    'sw',
    'bg',
    'bs',
    'ro',
    'pt',

#     'zho',
#     'fr',
#     'ko',
]

assert len(LANG2_list) == len(LANG3_list)

In [6]:
# vendor info here:  https://docs.google.com/spreadsheets/d/17zyMnZf2DOGzmte0qarBDIjyXAfrB24yoqwf5IknRc0/edit#gid=192447989
vendor_dict = {
    'rus':'Moravia',
    'deu':'Moravia',
    'hau':'Moravia',
    'isl':'Moravia',
    'ces':'Moravia',
    'jpn':'Moravia',
    'snd':'Moravia',
    'azj':'Lionbridge',
    'amh':'Moravia',
    'zul':'Lionbridge',
    'kat':'Moravia',
    'urd':'Lionbridge',
    'ara':'Moravia',
    'hin':'Lionbridge',
    'slv':'Moravia',
    'swh':'Lionbridge',
    'bul':'Moravia',
    'bos':'Lionbridge',
    'ron':'Moravia',
    'por':'Lionbridge',
    'asm':'Lionbridge',
    'umb':'Moravia',
    'tel':'Lionbridge',
    'kea':'Moravia',
    'ceb':'Moravia',
}

# 
# vendor

In [7]:
for L_key in zip(LANG2_list,LANG3_list):
    L2 = L_key[0]
    L3 = L_key[1]
    vendor = vendor_dict[L3].upper()
    print("Langauge code - 3 letter:",L3, "    2 letter:", L2, "   Vendor: ",vendor)
    
    
    # Import and built the FLORES Human Refence Set

    filename_v1_IDs = "/private/home/dlicht/flores101_data/2021_09_human_eval_sample_building/daiquery-unique flores IDs-2021-09-08 11_00am.tsv"
    df_ID_v1 = pd.read_csv(filename_v1_IDs, 
                              delimiter='\t', header=0, names=['FLORES_ID','s_verif','count'], quoting=3)
    df_ID_v1 = df_ID_v1[['FLORES_ID','s_verif']]
    df_ID_v1

    filename_eng = "/private/home/dlicht/flores101_data/FLORES_devtest/eng.devtest"
    df_F101_eng = pd.read_csv(filename_eng, 
                              delimiter='\t', header=None, names=['sentence1'], quoting=3)
    df_F101_eng.index.rename('FLORES_ID', inplace=True) 
    df_F101_eng['sentence1_lang'] = 'eng'
    # df_F101_eng['eng'] = df_F101_eng['eng'].str.replace('\"','').str.replace("\'",'')
    df_F101_eng

    df_F101 = pd.read_csv(f"/private/home/dlicht/flores101_data/FLORES_devtest/{L3}.devtest", 
                              delimiter='\t', header=None, names=['sentence2'],  quoting=3)
    df_F101['sentence2_lang'] = L3
    df_F101.index.rename('FLORES_ID', inplace=True)
    df_F101

    df_F101_J = pd.merge(df_F101_eng, df_F101, how='outer', on='FLORES_ID')
    df_F101_J

    df_F101_J[df_F101_J['sentence1'].str.contains('British')]



    HT_v1_ID_list = list(df_ID_v1.FLORES_ID)
    df_F101_v1_eng   = df_F101_eng[df_F101_eng.index.isin(df_ID_v1.FLORES_ID)]
    df_F101_v1       = df_F101[df_F101.index.isin(HT_v1_ID_list)]

    df_F101_v1_J = df_F101_v1_eng.merge(df_F101_v1, how='outer', on='FLORES_ID')
    df_F101_v1_J



    prop_sample = 0.5**0.5   # 1/sqrt(2)
    prop_sample

    sample_size_HT = int(df_F101_J.shape[0] * prop_sample)
    sample_size_HT

    sample_size_v1 = df_F101_v1_J.shape[0]
    sample_size_v1

    sample_size_suppletmentary_HT = sample_size_HT - sample_size_v1
    sample_size_suppletmentary_HT

    df_F101_v1_pool       = df_F101_J[   df_F101_J.index.isin(HT_v1_ID_list) ]
    df_F101_untapped_pool = df_F101_J[ ~ df_F101_J.index.isin(HT_v1_ID_list) ]
    print(df_F101_v1_pool.shape)
    df_F101_untapped_pool

    df_HT_sup_sample = df_F101_untapped_pool.sample(n=sample_size_suppletmentary_HT, replace=False)
    df_HT_sup_sample.sort_values('FLORES_ID')

    df_HT_sample = pd.concat([df_F101_v1_pool, df_HT_sup_sample])
    df_HT_sample

    # verification step
    # This will throw an error if the HT sample size is wrong
    assert df_HT_sample.shape[0] == sample_size_HT

    df_HT_sample_forward = df_HT_sample.copy()
    df_HT_sample_forward['direction'] = 'forward'
    df_HT_sample_forward

    df_HT_sample_backward = df_HT_sample.copy()
    df_HT_sample_backward['direction'] = 'backward'
    df_HT_sample_backward.rename(columns={
                                        "sentence1":"sentence2",
                                        "sentence2":"sentence1",
                                        "sentence1_lang":"sentence2_lang",
                                        "sentence2_lang":"sentence1_lang",
                                        }, inplace=True)
    df_HT_sample_backward

    df_HT_sample_paired = pd.concat([df_HT_sample_forward, df_HT_sample_backward]).sort_values(['FLORES_ID','sentence1_lang','sentence2_lang'])
    # create a hash to be our new ID
    # df_HT_sample_paired['fbid'] = df_HT_sample_paired.apply(lambda x: hash(tuple(x)),axis=1)
    df_HT_sample_paired['model']  = 'human'
    df_HT_sample_paired['dataset']='devtest'
    df_HT_sample_paired = df_HT_sample_paired[['sentence1_lang', 'sentence2_lang','sentence1', 'sentence2', 
           'model','direction','dataset']].reset_index()
    df_HT_sample_paired

    # Import the Machine Translations

    L2

    ### XX into EN

    filename_eng = f"/private/home/dlicht/flores101_data/m2m100_generations_21_10_09/{L2}_en_beam4.hyp"
    df_MT_into_EN = pd.read_csv(filename_eng, 
                              delimiter='\t', header=None, names=['sentence2'], quoting=3)
    df_MT_into_EN.index.rename('FLORES_ID', inplace=True) 
    df_MT_into_EN['sentence2_lang'] = 'eng'
    df_MT_into_EN

    df_F101_S = df_F101.rename(columns={'sentence2':'sentence1','sentence2_lang':'sentence1_lang'})
    df_F101_S

    df_MT_sample_backward = pd.merge(df_F101_S, df_MT_into_EN, how='right', on='FLORES_ID')
    df_MT_sample_backward['direction'] = 'backward'
    df_MT_sample_backward

    ### EN into XX

    filename = f"/private/home/dlicht/flores101_data/m2m100_generations_21_10_09/en_{L2}_beam4.hyp"
    df_MT_from_EN = pd.read_csv(filename, 
                              delimiter='\t', header=None, names=['sentence2'], quoting=3)
    df_MT_from_EN.index.rename('FLORES_ID', inplace=True) 
    df_MT_from_EN['sentence2_lang'] = L3
    df_MT_from_EN

    df_F101_eng

    df_MT_sample_forward = pd.merge(df_F101_eng, df_MT_from_EN, how='right', on='FLORES_ID')
    df_MT_sample_forward['direction'] = 'forward'
    df_MT_sample_forward

    df_MT_sample_paired = pd.concat([df_MT_sample_forward, df_MT_sample_backward]).sort_values(['FLORES_ID','sentence1_lang','sentence2_lang'])
    # create a hash to be our new ID
    # df_HT_sample_paired['fbid'] = df_HT_sample_paired.apply(lambda x: hash(tuple(x)),axis=1)
    df_MT_sample_paired['model']  = 'm2m100'
    df_MT_sample_paired['dataset']='devtest'
    df_MT_sample_paired = df_MT_sample_paired[['sentence1_lang', 'sentence2_lang','sentence1', 'sentence2', 
           'model','direction','dataset']].reset_index()
    df_MT_sample_paired



    # Import the Cross-Lingual Calibration Set

    filename = "Cross_Lingual_Calibration_Set_v1.tsv"
    write_dir = "/private/home/dlicht/fairseq-py/examples/nllb/human_eval_sample_generation/"

    df_CLC = pd.read_csv(write_dir+filename, delimiter='\t')
    df_CLC.drop(columns=['model','backtranslation_lang',
           'sampled_quality_score', 'quality_source'], inplace=True)
    df_CLC['model'] = 'Cross_Lingual_Calibration'
    df_CLC['direction'] = 'backward_calibration'
    df_CLC

    # Combine the 3 sets together

    df_sample_union = pd.concat([df_HT_sample_paired, df_MT_sample_paired, df_CLC])
    df_sample_union

    df_HT_sample_paired.groupby('model').count()

    df_sample_union['fbid'] = pd.util.hash_pandas_object(df_sample_union)  # hash_pandas_object yields only positive hashesdf_HT_sample_paired = 
    df_sample_union.tail(2)

    # throw an error if there are any duplicat fbid hashes
    print('max items with same fbid=',df_sample_union.groupby('fbid').count().max().max())
    assert df_sample_union.groupby('fbid').count().max().max() == 1



    # Randomize the Sample Order

    # randomize the sample
    df_final_sample = df_sample_union.sample(frac=1, replace=False)
    df_final_sample.set_index('fbid', inplace=True)
    df_final_sample

    # Write to Disk

    write_dir



    #write internal version
    filename = f"20210913_eval_NLLB_m2m100_{L3}.tsv"
    write_dir = '/private/home/dlicht/fairseq-py/examples/nllb/human_eval_sample_generation/master_copy/'

    if write_tsv:
        print(f"writing file {filename}")
        df_final_sample.to_csv(write_dir+filename, sep='\t',)

    df_final_sample.columns

    df_vendor_subset = df_final_sample[['sentence1_lang', 'sentence2_lang', 'sentence1',
           'sentence2']]
    df_vendor_subset

    # write vendor version
    filename = f"20210913_{vendor}_ADHOC_NLLB-HE_H2-R1_2021_{L3}.tsv"
    write_dir = '/private/home/dlicht/fairseq-py/examples/nllb/human_eval_sample_generation/vendor_copy/'

    if write_tsv:
        print(f"writing file {filename}")
        df_vendor_subset.to_csv(write_dir+filename, sep='\t',)

Langauge code - 3 letter: snd     2 letter: sd    Vendor:  MORAVIA
(630, 4)
max items with same fbid= 1
writing file 20210913_eval_NLLB_m2m100_snd.tsv
writing file 20210913_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_snd.tsv
Langauge code - 3 letter: azj     2 letter: az    Vendor:  LIONBRIDGE
(630, 4)
max items with same fbid= 1
writing file 20210913_eval_NLLB_m2m100_azj.tsv
writing file 20210913_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_azj.tsv
Langauge code - 3 letter: amh     2 letter: am    Vendor:  MORAVIA
(630, 4)
max items with same fbid= 1
writing file 20210913_eval_NLLB_m2m100_amh.tsv
writing file 20210913_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_amh.tsv
Langauge code - 3 letter: zul     2 letter: zu    Vendor:  LIONBRIDGE
(630, 4)
max items with same fbid= 1
writing file 20210913_eval_NLLB_m2m100_zul.tsv
writing file 20210913_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_zul.tsv
Langauge code - 3 letter: kat     2 letter: ka    Vendor:  MORAVIA
(630, 4)
max items with same fbid= 1
writing file 20210913_eval_N